In [237]:
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi, SNLE
from sbi.utils.posterior_ensemble import NeuralPosteriorEnsemble
import numpy as np
import moments
from sbi.utils import process_prior


from matplotlib import pyplot as plt
import dill
import os
import seaborn as sns
from joblib import Parallel, delayed
from sbi.inference import infer
from torch.distributions import MultivariateNormal
from torch.distributions import Independent

from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi import analysis as analysis

In [132]:
# Simulator
def momment_sim(prior):
    
    moment_data =  moments.Spectrum(moments.LinearSystem_1D.steady_state_1D(100, gamma=prior.detach().cpu().numpy(), theta=100.0))  # returns a masked array
    # masked arrays are objects and data is accessed through .data attribute or valid data through .compressed()
    actual_fs = moment_data.compressed()  
    x = torch.poisson(torch.tensor(actual_fs))
    return x.view(1,-1).type(torch.float32)

def true_data(prior):
    
    if prior.shape[0] > 1:
        moment_data =  moments.Spectrum(moments.LinearSystem_1D.steady_state_1D(100, gamma=prior[0].detach().cpu().numpy(), theta=100.0))
        actual_fs = moment_data.compressed()  
        x = torch.poisson(torch.tensor(actual_fs).repeat(prior.shape[0],1))
    else:
        moment_data =  moments.Spectrum(moments.LinearSystem_1D.steady_state_1D(100, gamma=prior.detach().cpu().numpy(), theta=100.0))
        actual_fs = moment_data.compressed()  
        x = torch.poisson(torch.tensor(actual_fs))
    return x, actual_fs


In [258]:
# Generate Prior
boxprior = utils.BoxUniform(low=1 * torch.ones(1), high=100 * torch.ones(1), device='cuda')
gamma_prior = torch.distributions.Gamma(100.0*torch.ones((1,),device='cuda'), 10.0*torch.ones((1,),device='cuda'))
#prior, *_ = process_prior(gamma_prior)
ind_gamma_prior = Independent(gamma_prior,reinterpreted_batch_ndims=1)
pop_dim = 100
prior=ind_gamma_prior


In [228]:
num_dim = 2
prior_mean = torch.zeros(num_dim)
prior_cov = torch.eye(num_dim)
Normprior = MultivariateNormal(loc=prior_mean, covariance_matrix=prior_cov)

In [229]:
Normprior._event_shape

torch.Size([2])

In [220]:
gamma_prior.sample((1,))

tensor([[8.9235]], device='cuda:0')

In [222]:
gamma_prior._event_shape

torch.Size([])

In [240]:
ind_gamma_prior._event_shape

torch.Size([1])

In [223]:
gamma_prior._batch_shape

torch.Size([1])

In [221]:
prior.sample((1,))

tensor([[9.9889]], device='cuda:0')

In [142]:
boxprior.sample((1,))

tensor([[67.5278]], device='cuda:0')

In [259]:
# Train SNLE
#simulator, prior = prepare_for_sbi(momment_sim, prior)
inferer = SNLE(prior, show_progress_bars=True, device='cuda', density_estimator="maf")


In [260]:
# Obtain posterior samples for different number of iid "observed" xos.
num_sim = 500
num_iid_trials = num_sim # if not num_sim it will not run

# Generate IID samples from the same prior value
theta_o = prior.sample((1,))
print("theta for true: {}".format(theta_o))
true_x, actual_fs = true_data(theta_o.repeat(num_iid_trials,1))
true_x = true_x.cuda()

theta for true: tensor([[10.8689]], device='cuda:0')


In [271]:
#posterior parameters
vi_parameters = dict(q="maf")
vi_parameters_2 = dict(q="nsf")

rounds = 100


In [243]:
true_x.shape

torch.Size([500, 99])

In [276]:
from sbi.inference import likelihood_estimator_based_potential
from sbi.inference.posteriors.vi_posterior import VIPosterior

posteriors = []
for i in range(0,rounds):
    
    if i == 0:
        theta, x = simulate_for_sbi(momment_sim, prior, num_sim)
        liklihood_estimator = inferer.append_simulations(theta, x).train(training_batch_size=50)
        potential_fn, theta_transform = likelihood_estimator_based_potential(liklihood_estimator, prior,x_o=true_x[0])
        a_post = VIPosterior(potential_fn=potential_fn,
                        theta_transform=theta_transform,
                        prior=prior, vi_method="IW", q="nsf")
        posteriors.append(a_post)
        proposal = a_post.train(max_num_iters=40, quality_control=False, K=50)
    else:
        print(proposal)
        theta, x = simulate_for_sbi(momment_sim, proposal, num_sim)
        
        liklihood_estimator = inferer.append_simulations(theta, x).train(training_batch_size=50)
        potential_fn, theta_transform = likelihood_estimator_based_potential(liklihood_estimator, proposal,x_o=true_x[0])
        a_post = VIPosterior(potential_fn=potential_fn,
                        theta_transform=theta_transform,
                        prior=prior, vi_method="IW", q="nsf")
        posteriors.append(a_post)
        proposal = a_post.train(max_num_iters=40, quality_control=False, K=50)


Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

 Neural network successfully converged after 21 epochs.

  0%|          | 0/40 [00:00<?, ?it/s]

Posterior conditional density p(θ|x) of type VIPosterior. It provides Variational inference to .sample() from the posterior and can evaluate the _normalized_ posterior density with .log_prob().


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [249]:
inferer = SNPE(prior, show_progress_bars=True, device='cuda', density_estimator="maf")
posteriors = []
proposal = prior
for i in range(0,rounds):
    

    theta, x = simulate_for_sbi(momment_sim, prior, num_sim)
    liklihood_estimator = inferer.append_simulations(theta, x, proposal=proposal).train(training_batch_size=50)
    posterior = inferer.build_posterior(density_estimator=liklihood_estimator, sample_with = "vi", 
                                        vi_method="fKL", vi_parameters=vi_parameters)
    posteriors.append(posterior)
    proposal = posterior.set_default_x(true_x[0]).train(max_num_iters=40, quality_control=False )



/mnt/sda/home/ludeep/Desktop/PopGen/FinBank/moments_sfs/momentssfs/lib/python3.10/site-packages/sbi/utils/torchutils.py:27: UserWarning: GPU was selected as a device for training the neural network. Note that we expect **no** significant speed ups in training for the default architectures we provide. Using the GPU will be effective only for large neural networks with operations that are fast on the GPU, e.g., for a CNN or RNN `embedding_net`.
  warnings.warn(


Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

/mnt/sda/home/ludeep/Desktop/PopGen/FinBank/moments_sfs/momentssfs/lib/python3.10/site-packages/sbi/utils/user_input_checks.py:702: UserWarning: Data x has device 'cpu'.Moving x to the data_device 'cuda:0'.Training will proceed on device 'cuda:0'.
  warnings.warn(
/mnt/sda/home/ludeep/Desktop/PopGen/FinBank/moments_sfs/momentssfs/lib/python3.10/site-packages/sbi/neural_nets/flow.py:141: UserWarning: In one-dimensional output space, this flow is limited to Gaussians
  warn("In one-dimensional output space, this flow is limited to Gaussians")


 Neural network successfully converged after 25 epochs.

/mnt/sda/home/ludeep/Desktop/PopGen/FinBank/moments_sfs/momentssfs/lib/python3.10/site-packages/pyro/nn/auto_reg_nn.py:179: UserWarning: ConditionalAutoRegressiveNN input_dim = 1. Consider using an affine transformation instead.
  warnings.warn(


...

Using SNPE-C with atomic loss
 Neural network successfully converged after 22 epochs.

  0%|          | 0/40 [00:00<?, ?it/s]

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

Using SNPE-C with atomic loss
 Training neural network. Epochs trained: 10

KeyboardInterrupt: 

In [253]:
from sbi.inference import SNRE_A

inferer = SNRE_A(prior, show_progress_bars=True, device='cuda')
posteriors = []
proposal = prior
for i in range(0,rounds):
    

    theta, x = simulate_for_sbi(momment_sim, proposal, num_sim)
    liklihood_estimator = inferer.append_simulations(theta, x).train(training_batch_size=50)
    posterior = inferer.build_posterior(density_estimator=liklihood_estimator, sample_with = "vi", 
                                        vi_method="fKL", vi_parameters=vi_parameters)
    posteriors.append(posterior)
    proposal = posterior.set_default_x(true_x[0]).train(max_num_iters=40, quality_control=False )

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

 Neural network successfully converged after 38 epochs.

  0%|          | 0/40 [00:00<?, ?it/s]

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

 Neural network successfully converged after 35 epochs.

...

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

 Neural network successfully converged after 21 epochs.

  0%|          | 0/40 [00:00<?, ?it/s]

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

RuntimeError: invalid Poisson rate, expected rate to be non-negative

In [255]:
from sbi.utils import RestrictionEstimator

restriction_estimator = RestrictionEstimator(prior=prior, device='cuda')
proposals = [prior]

for i in range(0,rounds):
    theta, x = simulate_for_sbi(momment_sim, proposals[-1], 500)
    restriction_estimator.append_simulations(theta, x)
    if (i < rounds - 1):  # training not needed in last round because classifier will not be used anymore.
        classifier = restriction_estimator.train()
    proposals.append(restriction_estimator.restrict_prior())

all_theta, all_x, _ = restriction_estimator.get_simulations()

inferer = SNRE_A(prior, show_progress_bars=True, device='cuda')
density_estimator = inferer.append_simulations(all_theta, all_x).train()
posterior = inferer.build_posterior(density_estimator, sample_with = "vi",
                                    vi_method="fKL", vi_parameters=vi_parameters)
inferer.set_default_x(true_x[0]).train(max_num_iters=40, quality_control=False )


TypeError: RestrictionEstimator.__init__() got an unexpected keyword argument 'device'

In [98]:
posteriors = []
thetas= []
xs = []
for i in range(0,rounds):
    
    if i == 0:
        for j in range(0,num_sim): 
            a_theta = prior.sample((1,)).detach().cpu().numpy()
            x = momment_sim(a_theta)
            xs.append(x)
            thetas.append(a_theta)
        theta = torch.tensor(thetas, dtype=torch.float32)
        x = torch.stack(xs).type(torch.float32)
        liklihood_estimator = inferer.append_simulations(theta, x).train()
        a_post = inferer.build_posterior(density_estimator=liklihood_estimator, sample_with = "vi", vi_method="fKL", vi_parameters=vi_parameters)
        posteriors.append(a_post)
        proposal = a_post.set_default_x(true_x).train(max_num_iters=40, quality_control=False )
    else:
        for j in num_sim: 
            a_theta = prior.sample((1,)).detach().cpu().numpy()
            x = momment_sim(a_theta)
            xs.append(x)
            thetas.append(a_theta)
        theta = torch.tensor(thetas, dtype=torch.float32)
        x = torch.stack(xs).type(torch.float32)
        #theta, x = simulate_for_sbi(simulator, proposal, num_sim)
        liklihood_estimator = inferer.append_simulations(theta, x, proposal=proposal).train(training_batch_size=20)
        posteriors.append(a_post)
        proposal = a_post.set_default_x(true_x).train(max_num_iters=40, quality_control=False )

RuntimeError: output with shape [50, 50] doesn't match the broadcast shape [50, 50, 50]

In [238]:
# 2 rounds: first round simulates from the prior, second round simulates parameter set
# that were sampled from the obtained posterior.
num_rounds = 2
# The specific observation we want to focus the inference on.
x_o = torch.zeros(
    3,
)

num_dim = 3
prior = utils.BoxUniform(low=-2 * torch.ones(num_dim), high=2 * torch.ones(num_dim))

posteriors = []
proposal = prior


def linear_gaussian(theta):
    return theta + 1.0 + torch.randn_like(theta) * 0.1

simulator, prior = prepare_for_sbi(linear_gaussian, prior)
inference = SNPE(prior=prior)


for _ in range(num_rounds):
    theta, x = simulate_for_sbi(simulator, proposal, num_simulations=500)

    # In `SNLE` and `SNRE`, you should not pass the `proposal` to `.append_simulations()`
    density_estimator = inference.append_simulations(
        theta, x, proposal=proposal
    ).train()
    posterior = inference.build_posterior(density_estimator, sample_with = "vi", vi_method="fKL")
    posteriors.append(posterior)
    proposal = posterior.set_default_x(x_o).train(max_num_iters=40, quality_control=False )

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

 Neural network successfully converged after 83 epochs.

  0%|          | 0/40 [00:00<?, ?it/s]

Running 500 simulations.:   0%|          | 0/500 [00:00<?, ?it/s]

Using SNPE-C with atomic loss
 Neural network successfully converged after 33 epochs.

  0%|          | 0/40 [00:00<?, ?it/s]